   ![alt text](https://www.mbari.org/wp-content/uploads/2014/11/logo-mbari-3b.png "MBARI")

  <div align="left">Copyright (c) 2021, MBARI</div>
    
  * Distributed under the terms of the GPL License
  * Maintainer: dcline@mbari.org
  * Authors: Danelle Cline dcline@mbari.org, John Ryan ryjo@mbari.org

## Basic Exploration of the 2 kHz Pacific Ocean Audio Data in the AWS Open Data Registry

---
An extensive (5+ years and growing) archive of sound recordings from a deep-sea location [along the eastern margin of the North Pacific Ocean](https://www.mbari.org/at-sea/cabled-observatory/) has been made available through AWS Open data.  Temporal coverage of the recording archive has been 95% since project inception in July 2015.  The original recordings have a sample rate of 256 kHz.  For many research applications it is convenient to work with data having a lower sample rate.  This notebook illustrates basic methods to access and process a calibrated spectrogram from the decimated 2 kHz audio archive.

If you use this data set, please **[cite our project](https://ieeexplore.ieee.org/document/7761363).**


## Data Overview
The decimated audio data are in [WAV](https://en.wikipedia.org/wiki/WAV) format in an s3 bucket named <b>pacific-sound-2khz</b>.  They are further organized by year and month.  Buckets are stored as objects, so the data isn't physically stored in folders or directories as you may be famaliar with, but you can think of it conceptually as follows:

```
pacific-sound-2khz
      |
      ----2020
        |
        |----01
        ...
        |----12
```


## Install dependencies
First, let's install dependencies and include all packages used in this tutorial. This only needs to be done once for the duration of this notebook.

In [ ]:
!pip install -q boto3
!pip install -q soundfile
!pip install -q scipy
!pip install -q numpy
!pip install -q matplotlib
import boto3, botocore
from botocore import UNSIGNED
from botocore.client import Config
from six.moves.urllib.request import urlopen
import io
import scipy
from scipy import signal
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

## List the contents of a monthly directory

In [ ]:
s3 = boto3.client('s3',
    aws_access_key_id='',
    aws_secret_access_key='', 
    config=Config(signature_version=UNSIGNED))

In [ ]:
year = "2020"
month = "01"
bucket = 'pacific-sound-2khz'

for obj in s3.list_objects_v2(Bucket=bucket, Prefix=f'{year}/{month}')['Contents']:
    print(obj['Key'])

## Retrieve metadata for a file

In [ ]:
year = "2020"
month = "01"
filename = 'MARS-20200101T000000Z-2kHz.wav'
bucket = 'pacific-sound-2khz'
key = f'{year}/{month}/{filename}'

url = f'https://{bucket}.s3.amazonaws.com/{key}'

sf.info(io.BytesIO(urlopen(url).read(20_000)), verbose=True) 

## Calibrated Spectrum Levels

### Produce calibrated spectrogram for a full day
For the low-frequency (2 kHz) data, calibration data are not frequency dependent; a single low-frequency calibration value is used.  Its value depends on time of data collection, as two hydrophones have been deployed sequentially at the same site.  Before 14 June 2017, the calibration value is -168.8 dB re V / uPa (measured at 26 Hz).  After this date the value is -177.9 dB re V / uPa (measured at 250 Hz).  See also:


*   https://bitbucket.org/mbari/pacific-sound/src/master/MBARI_MARS_Hydrophone_Deployment01.json
*   https://bitbucket.org/mbari/pacific-sound/src/master/MBARI_MARS_Hydrophone_Deployment02.json




In [ ]:
# create a 1-Hz x n second calibrated spectrogram at 1 second resolution
print(f'Reading from {url}')
x, sample_rate = sf.read(io.BytesIO(urlopen(url).read()),dtype='float32') 
v = x*3   # convert scaled voltage to volts
v.shape, v.size, sample_rate
a = np.arange(v.size)+1
# define segment processing
nsec = (v.size)/sample_rate # number of seconds in vector
spa = 60  # seconds per average
nseg = int(nsec/spa)
print(f'{nseg} segments of length {spa} seconds in {nsec} seconds of audio')

In [ ]:
# initialize empty LTSA
nfreq = int(sample_rate/2+1)
nfreq,nseg
LTSA = np.empty((nfreq, nseg), float)
LTSA.shape

In [ ]:
# get window for welch
w = scipy.signal.get_window('hann',sample_rate)

# process LTSA
for x in range(0,nseg):
  cstart = x*spa*sample_rate
  cend = (x+1)*spa*sample_rate
  f,psd = scipy.signal.welch(v[cstart:cend],fs=sample_rate,window=w,nfft=sample_rate)
  psd = 10*np.log10(psd) + 177.9
  LTSA[:,x] = psd

In [ ]:
nsec, nseg, LTSA.shape

### Plot the calibrated spectrogram
Note: The sharp drop in signal approaching 1 kHz reflects the attributes of the decimation filter applied to produce the 2 kHZ data from the original 256 kHz data.

In [ ]:
plt.figure(dpi=300)
im = plt.imshow(LTSA,aspect='auto',origin='lower',vmin=30,vmax=100)
plt.yscale('log')
plt.ylim(10,1000)
plt.colorbar(im)
plt.xlabel('Minute of day')
plt.ylabel('Frequency (Hz)')
plt.title('Calibrated spectrum levels')